In [1]:
import pandas as pd
import re
from DataCollector import DataCollector
DATA_DIR = "./src/data"

In [2]:
collector = DataCollector(DATA_DIR)

In [3]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, XSD

In [4]:
# Criar o modelo (grafo RDF)
model = Graph()

# Definir namespaces 
base = Namespace("http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#")
xsd  = Namespace("http://www.w3.org/2001/XMLSchema#")
foaf = Namespace("http://xmlns.com/foaf/0.1/")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
rdf  = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
dbo  = Namespace("http://dbpedia.org/ontology/")
dbp  = Namespace("http://dbpedia.org/property/")
dbr  = Namespace("http://dbpedia.org/resource/")

# Vincular os namespaces ao grafo 
model.bind("", base)
model.bind("xsd", xsd)
model.bind("foaf", foaf)
model.bind("rdf", rdf)
model.bind("rdfs", rdfs)
model.bind("dbo", dbo)
model.bind("dbp", dbp)
model.bind("dbr", dbr)

In [5]:
def add_countries_to_model():
    countries = collector.get_countries()

    for country_name in countries:
        country_name_formated = re.sub(r"[^a-zA-Z0-9]", "", country_name)
        country = URIRef(base + country_name_formated)
        model.add((country, RDF.type, dbo.Country))
        model.add((country, foaf.name, Literal(country_name)))

In [7]:
def add_styles_to_model():
    artist_styles = collector.get_artist_styles()
    work_styles = collector.get_work_styles()
    styles = set(artist_styles + work_styles)

    for style_name in styles:
        style_name_formated = "Style_"+re.sub(r"[^a-zA-Z0-9]", "", style_name)
        style = URIRef(base + style_name_formated)
        model.add((style, RDF.type, base.Style))
        model.add((style, foaf.name, Literal(style_name)))

In [8]:
def add_subjects_to_model():
    subjects = collector.get_subjects()

    for subject_name in subjects:
        subject_name_formated = re.sub(r"[^a-zA-Z0-9]", "", subject_name)
        subject = URIRef(base + subject_name_formated)
        model.add((subject, RDF.type, base.Subject))
        model.add((subject, foaf.name, Literal(subject_name)))

In [9]:
def add_nationalities_to_model():
    nationalities = collector.get_artist_nationalities()

    for nationality_name in nationalities:
        nationality_name_formated = re.sub(r"[^a-zA-Z0-9]", "", nationality_name)
        nationality = URIRef(base + nationality_name_formated)
        model.add((nationality, RDF.type, base.Nationality))
        model.add((nationality, foaf.name, Literal(nationality_name)))

In [10]:
def add_museums_to_model():
    data = collector.get_museums()
    tam = data.shape[0]
    
    for i in range (tam):
        museum_identifier = data.iloc[i]["identifier"]
        museum = URIRef(base + museum_identifier)
        model.add((museum, RDF.type, dbo.Museum))

        museum_name = str(data.iloc[i]["name"])
        model.add((museum, foaf.name, Literal(museum_name)))

        country_name = re.sub(r"[^a-zA-Z0-9]", "", str(data.iloc[i]["country"]))
        if country_name != "nan":
            model.add((museum, dbp.country, URIRef(base + country_name )))

        state = str(data.iloc[i]["state"])
        if state != "nan":
            model.add((museum, base.state, Literal(state)))
        
        city = str(data.iloc[i]["city"])
        if city != "nan":
            model.add((museum, base.city, Literal(city)))
        
        address = str(data.iloc[i]["address"])
        if address != "nan":
            model.add((museum, base.address, Literal(address)))
        
        postal = str(data.iloc[i]["postal"])
        if postal != "nan":
            model.add((museum, base.postal, Literal(postal)))
        
        phone = str(data.iloc[i]["phone"])
        if phone != "nan":
            model.add((museum, base.phone, Literal(phone)))
        
        url = str(data.iloc[i]["url"])
        if url != "nan":
            model.add((museum, foaf.homepage, Literal(url)))

In [11]:
def add_artists_to_model():
    data = collector.get_artists()
    tam = data.shape[0]
    
    for i in range (tam):
        artist_identifier = data.iloc[i]["identifier"]
        artist = URIRef(base + artist_identifier)
        model.add((artist, RDF.type, dbo.Artist))

        full_name = str(data.iloc[i]["full_name"])
        model.add((artist, foaf.name, Literal(full_name)))

        first_name = str(data.iloc[i]["first_name"])
        if first_name != "nan":
            model.add((artist, foaf.firstName, Literal(first_name)))

        last_name = str(data.iloc[i]["last_name"])
        if last_name != "nan":
            model.add((artist, foaf.lastName, Literal(last_name)))
        
        birth = str(data.iloc[i]["birth"])
        if birth != "nan":
            model.add((artist, base.birth, Literal(birth)))

        death = str(data.iloc[i]["death"])
        if death != "nan":
            model.add((artist, base.death, Literal(death)))
        
        nationality = re.sub(r"[^a-zA-Z0-9]", "", str(data.iloc[i]["nationality"]))
        if nationality != "nan":
            model.add((artist, dbp.nationality, URIRef(base + nationality )))
        
        style = re.sub(r"[^a-zA-Z0-9]", "", str(data.iloc[i]["style"]))
        if style != "nan":
            model.add((artist, dbo.movement, URIRef(base + style )))

In [12]:
def add_works_to_model():
    data = collector.get_works()
    tam = data.shape[0]

    for i in range (tam):
        work_identifier = data.iloc[i]["identifier"]
        work = URIRef(base + work_identifier)
        model.add((work, RDF.type, dbo.Painting))

        name = str(data.iloc[i]["name"])
        model.add((work, foaf.name, Literal(name)))

        artist = collector.get_artist_by_id(data.iloc[i]["artist_id"])
        if artist is not None:
            model.add((work, dbp.artist, URIRef(base + artist )))
        
        style = re.sub(r"[^a-zA-Z0-9]", "", str(data.iloc[i]["style"]))
        if style != "nan":
            model.add((work, dbp.style, URIRef(base + style )))
        
        subject = collector.get_subject_by_id(data.iloc[i]["work_id"])
        if subject is not None:
            model.add((work, dbp.subject, URIRef(base + subject )))

        museum_id = data.iloc[i]["museum_id"]
        if museum_id != "nan":
            work_museum = collector.get_museum_by_id(museum_id)
            if work_museum is not None:
                model.add((work, dbp.museum, URIRef(base + work_museum)))
        

In [13]:
add_nationalities_to_model()
add_subjects_to_model()
add_countries_to_model()
add_styles_to_model()
add_works_to_model()
add_museums_to_model()
add_artists_to_model()

In [14]:
# Serializar o grafo para um arquivo ou string
output_file = "output.ttl"  
with open(output_file, "w", encoding="utf-8") as fout:
    fout.write(model.serialize(format="turtle"))

In [15]:
from owlrl import DeductiveClosure, RDFS_Semantics

In [16]:
schema_file = "./paintingsontology.ttl"
data_file = "./output.ttl"

schema = Graph()
schema.parse(schema_file)  # Ler o esquema

data = Graph()
data.parse(data_file, format="turtle")  # Ler os dados 

inferred_model = Graph()
inferred_model += schema
inferred_model += data

DeductiveClosure(RDFS_Semantics).expand(inferred_model)  # Inferência RDFS

print("Modelo inferido:")
output_file2 = "output2.ttl"  
with open(output_file2, "w", encoding='utf-8') as fout:
    fout.write(inferred_model.serialize(format="turtle"))

Modelo inferido:


In [55]:
museums = collector.get_museums()


In [56]:
museums_name = museums["name"].tolist()

In [57]:
from geopy.geocoders import Nominatim
import random

def obter_lat_lon_nominatim(endereco):
    geolocator = Nominatim(user_agent="sua_aplicacao")
    localizacao = geolocator.geocode(endereco)
    if localizacao:
        return localizacao.latitude, localizacao.longitude
    else:
        print("Endereço não encontrado")
        return None, None

# Exemplo de uso
elemento_aleatorio = random.choice(museums_name)
endereco = random.choice(museums_name)
latitude, longitude = obter_lat_lon_nominatim(endereco)
print(f"Latitude: {latitude}, Longitude: {longitude}")

import folium

# create tuples representing our location
location = float(latitude), float(longitude)

# center the map 
center = (latitude, longitude)

# create a Folium map centred at the above location
m = folium.Map(location=center, zoom_start=50, width=800, height=400)

# add markers at the locations
folium.Marker(location, popup="The postcode brought me here").add_to(m)

# refer to the map to display it in Jupyter/Colab notebooks
m

Latitude: 48.85941225, Longitude: 2.2672551197388042


In [17]:
# Pesquisando dados de artistas por inicio do nome
searchQuery = 'Ad'.lower()
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?artistName ?birth ?death ?style ?nationality
        WHERE {{
            ?artist
                rdf:type dbo:Artist ;
                foaf:name ?artistName ;
            .
            OPTIONAL {{?artist :birth ?birth .}}
            OPTIONAL {{?artist :death ?death .}}
            OPTIONAL {{?artist dbo:movement/foaf:name ?style .}}
            OPTIONAL {{?artist dbp:nationality/foaf:name ?nationality .}}
            FILTER (STRSTARTS(LCASE(STR(?artistName)), "{searchQuery}"))
        }}
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
        'dbo': dbo,
        'dbp': dbp,
    }
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

Adolph Ulrich Wertmuller | 1751 | 1811 | NULL | Swiss
Adèle Romany | 1769 | 1846 | NULL | French
Adriaan De Lelie | 1755 | 1820 | NULL | Dutch
Adélaïde Labille-Guiard | 1749 | 1803 | NULL | French
Adriaen Thomasz. Key | 1544 | 1589 | NULL | Dutch


In [18]:
# Pesquisando dados de museus por inicio do nome
searchQuery = 'Mus'.lower()
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?museumName ?country ?state ?city ?address ?postal ?phone ?url
        WHERE {{
            ?museum
                rdf:type dbo:Museum ;
                foaf:name ?museumName ;
            .
            OPTIONAL {{?museum dbo:country/foaf:name ?country .}}
            OPTIONAL {{?museum :state ?state .}}
            OPTIONAL {{?museum :city ?city .}}
            OPTIONAL {{?museum :address ?address .}}
            OPTIONAL {{?museum :postal ?postal .}}
            OPTIONAL {{?museum :phone ?phone .}}
            OPTIONAL {{?museum foaf:homepage ?url .}}
            FILTER (STRSTARTS(LCASE(STR(?museumName)), "{searchQuery}"))
        }}
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
        'dbo': dbo,
        'dbp': dbp,
    }
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

Museum of Fine Arts, Houston | NULL | TX | Houston | 1001 Bissonnet St | 77005 | +1 713 639-7300 | https://www.mfah.org/
Museum of Fine Arts | NULL | NULL | Bern | Hodlerstrasse 8 | 3011 | +41 31 328 09 44 | https://www.kunstmuseumbern.ch/en/
Musée des Beaux-Arts Nancy | NULL | NULL | Nancy | 3 Pl. Stanislas | 54000 | +33 3 83 85 30 01 | https://useum.org/museum/Museum-of-Fine-Arts-of-Nancy
Musée du Louvre | NULL | Paris | 75001 | Rue de Rivoli | NULL | 33 1 40 20 50 50 | https://www.louvre.fr/en
Musée des Beaux-Arts Quimper | NULL | Quimper | 29000 | 40 Pl. Saint-Corentin | NULL | 33 2 98 95 45 20 | https://www.mbaq.fr/en/home-3.html
Museum Folkwang | NULL | Essen | 45128 | Museumsplatz 1 | NULL | 49 201 8845000 | https://www.museum-folkwang.de/en
Museo Thyssen-Bornemisza | NULL | NULL | Madrid | P.º del Prado, 8 | 28014 | + 34 917 91 13 70 | https://www.museothyssen.org/en
Museum Legion of Honor | NULL | CA | San Francisco | 100 34th  Avenue | 94121 | +1 415 750-3600 | https://www.fa

In [20]:
# Pesquisando dados de obras por inicio do nome
searchQuery = 'The Grand'.lower()
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?workName ?artist ?style ?subject ?location
        WHERE {{
            ?work
                rdf:type dbo:Painting ;
                foaf:name ?workName ;
            .
            OPTIONAL {{?work dbp:artist/foaf:name ?artist .}}
            OPTIONAL {{?work dbp:style/foaf:name ?style .}}
            OPTIONAL {{?work dbp:subject/foaf:name ?subject .}}
            OPTIONAL {{?work dbp:museum/foaf:name ?location .}}
            FILTER (STRSTARTS(LCASE(STR(?workName)), "{searchQuery}"))
        }}
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
        'dbo': dbo,
        'dbp': dbp,
    }
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

The Grand Canal in Venice with Palazzo Bembo | Francesco Guardi | NULL | NULL | The J. Paul Getty Museum
The Grand Canal in Venice | Albert Marquet | NULL | NULL | NULL
The Grand Canal Blue Venice | Édouard Manet | NULL | NULL | NULL
The Grande Creuse at Pont de Vervy | Claude Monet | NULL | NULL | Philadelphia Museum of Art
The Grand Canal in Venice from Palazzo Flangini to Campo San Marcuola | Canaletto | NULL | NULL | The J. Paul Getty Museum
The Grand Canal, Venice | Édouard Manet | NULL | NULL | NULL
The Grand Canyon of the Colorado | Thomas Moran | NULL | NULL | NULL
The Grand Canal from Palazzo Flangini to Campo San Marcuola | Canaletto | NULL | NULL | NULL
The Grand Canal with San Simeone Piccolo, Venice | Canaletto | NULL | NULL | National Gallery
The Grand Canal Above the Rialto | Francesco Guardi | NULL | NULL | The Metropolitan Museum of Art
The Grand Canyon of the Yellowstone | Thomas Moran | NULL | NULL | NULL
The Grand Bath at Bursa | Jean-Léon Gérôme | NULL | NULL | NUL

In [21]:
# Pesquisando dados de estilos por inicio do nome
searchQuery = 'America'.lower()
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?styleName
        WHERE {{
            ?style
                rdf:type :Style ;
                foaf:name ?styleName ;
            .
            FILTER (STRSTARTS(LCASE(STR(?styleName)), "{searchQuery}"))
        }}
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
    }
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

American Landscape
American West
America West
American Art


In [22]:
# Pesquisando dados de temas por inicio do nome
searchQuery = 'A'.lower()
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?subjectName
        WHERE {{
            ?subject
                rdf:type :Subject ;
                foaf:name ?subjectName ;
            .
            FILTER (STRSTARTS(LCASE(STR(?subjectName)), "{searchQuery}"))
        }}
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
    }
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

Autumn/Fall
Abstract/Modern Art
Architectures


In [26]:
# Listar 10 obras de um museu junto com o estilo daquela obra
museumURL = 'http://www.clevelandart.org/'
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        SELECT ?title ?style
        WHERE {{
            SERVICE <https://dbpedia.org/sparql> {{
                ?workDbo a dbo:Work ;
                         dbp:title ?title ;
                         dbo:museum/dbp:website <{museumURL}> .
            }}
            OPTIONAL {{
                ?workLocal
                    rdf:type dbo:Painting ;
                    foaf:name ?titleLocal ;
                    dbp:style/foaf:name ?style ;
                .
                FILTER (STR(?title) = STR(?titleLocal))
            }}
        }}
        LIMIT 10
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
        'dbr': dbr,
        'dbo': dbo,
        'dbp': dbp,
    }
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

Bottle, Glass, Fork | NULL
Stag at Sharkey's | NULL
Constellations | NULL
The Large Plane Trees | NULL
Cleveland Apollo | NULL
Fall of Simon Magus | NULL
The Crucifixion of Saint Andrew | NULL
Un lièvre et un gigot de mouton | NULL
La Vie | NULL
Source | NULL


In [28]:
# Retornar os dados e a imagem de um artista pelo nome
artistName = 'Adélaïde Labille-Guiard'
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?birth ?death ?style ?nationality ?image
        WHERE {{
            OPTIONAL {{
                SERVICE <https://dbpedia.org/sparql> {{
                    ?artistDbo a dbo:Animal ;
                        rdfs:label "{artistName}"@en ;
                        dbo:thumbnail ?image .
                }}
            }}
            ?artist
                rdf:type dbo:Artist ;
                foaf:name "{artistName}" ;
            .
            OPTIONAL {{?artist :birth ?birth .}}
            OPTIONAL {{?artist :death ?death .}}
            OPTIONAL {{?artist dbo:movement/foaf:name ?style .}}
            OPTIONAL {{?artist dbp:nationality/foaf:name ?nationality .}}
        }}
        LIMIT 1
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
        'dbr': dbr,
        'dbo': dbo,
        'dbp': dbp,
        'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
    }
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

1749 | 1803 | NULL | French | http://commons.wikimedia.org/wiki/Special:FilePath/Adélaïde_Labille-Guiard_-_Self-Portrait_with_Two_Pupils_-_The_Metropolitan_Museum_of_Art.jpg?width=300


In [29]:
# Retornar os dados e as coordenadas de um museu pelo nome
museumName = 'National Maritime Museum'
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?country ?state ?city ?address ?postal ?phone ?url ?coords
        WHERE {{
            OPTIONAL {{
                SERVICE <https://dbpedia.org/sparql> {{
                    ?museumDbo a dbo:Building ;
                        rdfs:label "{museumName}"@en ;
                        georss:point ?coords .
                }}
            }}
            ?museum
                rdf:type dbo:Museum ;
                foaf:name "{museumName}" ;
            .
            OPTIONAL {{?museum dbp:country/foaf:name ?country .}}
            OPTIONAL {{?museum :state ?state .}}
            OPTIONAL {{?museum :city ?city .}}
            OPTIONAL {{?museum :address ?address .}}
            OPTIONAL {{?museum :postal ?postal .}}
            OPTIONAL {{?museum :phone ?phone .}}
            OPTIONAL {{?museum foaf:homepage ?url .}}
        }}
        LIMIT 1
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
        'dbr': dbr,
        'dbo': dbo,
        'dbp': dbp,
        'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
        'georss': 'http://www.georss.org/georss/',
    }
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

United Kingdom | SE10 9NF | London | Romney Rd | NULL | 44 20 8858 4422 | https://www.rmg.co.uk/national-maritime-museum | 51.481111 -0.005556
